## Error Analysis and Grad-CAM Interpretation

This notebook documents the **error analysis and interpretability workflow**
for the trained baseline CNN model.

To ensure stability and reproducibility:
- Model training and evaluation are performed in `02_baseline_cnn.ipynb`
- Grad-CAM visualizations and inference are executed via an external script
  (`scripts/run_gradcam.py`) to avoid GPU and kernel instability in notebooks

This notebook focuses on:
- Dataset reconstruction and label mapping
- Qualitative interpretation of Grad-CAM outputs
- Analysis of correct vs incorrect predictions


##### Imports and Setup

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

from src.data_loader import collect_image_paths, stratified_split
from src.models import build_baseline_cnn


##### Dataset Reconstruction and Label Encoding

In [ ]:
# Dataset and split
DATASET_PATH = "../data/raw/PlantVillage"
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 4

image_paths, labels = collect_image_paths(DATASET_PATH)
X_train, y_train, X_val, y_val, X_test, y_test = stratified_split(
    image_paths, labels
)

# Encode class labels
class_names = sorted(set(labels))
class_to_index = {name: idx for idx, name in enumerate(class_names)}
index_to_class = {idx: name for name, idx in class_to_index.items()}

y_test_encoded = np.array([class_to_index[y] for y in y_test])


: 

## Grad-CAM Execution Strategy

Inference and Grad-CAM visualization are executed via
`scripts/run_gradcam.py` instead of this notebook.

**Reason:**
- Prevents Jupyter kernel crashes during gradient-based visualization
- Enables stable batch-wise Grad-CAM generation
- Allows saving correct and incorrect predictions reproducibly

Generated outputs are stored under:
- `results/gradcam/correct/`
- `results/gradcam/incorrect/`


## Manual Grad-CAM Inspection — Observations

### Correct Predictions
- Heatmaps are localized on lesion regions
- Attention aligns with infected or discolored areas
- Minimal activation on background

### Incorrect Predictions
- Heatmaps activate background or leaf boundaries
- Attention focuses on veins or edges instead of lesions
- Confusions occur between visually similar diseases
  (e.g., Early blight vs Late blight)


## Error Summary

| Case Type | Observation | Interpretation |
|---------|------------|---------------|
| Correct | Heatmap focused on lesion regions | Model learned disease-specific cues |
| Incorrect | Activation on background | Weak localization / dataset bias |
| Incorrect | Confusion between similar diseases | High visual similarity across classes |


## Reference: Baseline Training Behavior

Training dynamics of the baseline CNN were analyzed in
`02_baseline_cnn.ipynb`.

Saved plots:
- `results/plots/baseline_accuracy_curve.png`
- `results/plots/baseline_loss_curve.png`

These plots confirm stable convergence prior to error analysis.
